# Deep Learning Toolkit for Splunk - Barebone Notebook

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [ ]:
import time
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Input 
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,LSTM,Dropout,Dense,Activation
from tensorflow.keras.activations import sigmoid
import matplotlib.pyplot as plt
import math
import numpy as np
import pickle
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing import sequence
import datetime
from collections import Counter
MODEL_DIRECTORY = "/srv/app/model/data/"
MAX_VOCAB = 40
pd.options.mode.chained_assignment = None 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("tensorflow version: " + tf.__version__)

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer mode=stage algo=barebone epochs=10 batch_size=1 s from feature_* into app:barebone_model

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [ ]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("dga_model_dltk")
print(df.describe())
print(param)

## Stage 2 - create and initialize a model

In [ ]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = tf.keras.models.load_model(MODEL_DIRECTORY + "pretrained_dga_apply_dga_model_dltk")
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(init(df,param))

## Stage 3 - fit the model

In [ ]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

In [ ]:
# apply your model
# returns the calculated results
tokenizer = pickle.load(open(MODEL_DIRECTORY+"pretrained_dga_apply_dga_model_dltk/tokenizer", 'rb'))
vectorizer_words = pickle.load(open(MODEL_DIRECTORY+"pretrained_dga_apply_dga_model_dltk/vectorizer_words", 'rb'))
vectorizer_non_dga_domains = pickle.load(open(MODEL_DIRECTORY+"pretrained_dga_apply_dga_model_dltk/vectorizer_non_dga_domains", 'rb'))
weight_words = pickle.load(open(MODEL_DIRECTORY+"pretrained_dga_apply_dga_model_dltk/weight_words", 'rb'))
weight_non_dga_grams = pickle.load(open(MODEL_DIRECTORY+"pretrained_dga_apply_dga_model_dltk/weight_non_dga_grams", 'rb'))
domains = pickle.load(open(MODEL_DIRECTORY+"pretrained_dga_apply_dga_model_dltk/domains", 'rb'))
model = tf.keras.models.load_model(MODEL_DIRECTORY + "pretrained_dga_apply_dga_model_dltk")

def entropy(domain):
        p, lns = Counter(domain), float(len(domain))
        return -sum( count/lns * math.log(count/lns, 2) for count in p.values())
    
alexa_domains = domains['domain']

    
def is_in_alexa1m(domain,domains):
    test_in_alexa_domains = set(alexa_domains) & domains
    return ((domain in test_in_alexa_domains))

    
    
def add_features(df):
    print ("1. Done adding ngram features")
    X_1= weight_words * vectorizer_words.transform(df['domain']).T 
    X_2= weight_non_dga_grams * vectorizer_non_dga_domains.transform(df['domain']).T
    X_3 =   df['domain'].map(lambda x: entropy(x)) 
    print ("2. Done adding entropy")
    X_4 = df['domain'].map(lambda x: len(x))
    print ("3. Done adding length of domain")
    domains = set(df['domain'])
    X_5 = df['domain'].map(lambda x: is_in_alexa1m(x,domains))
    print ("4. Done adding domain present in alexa domains")
    X_5 = X_5.astype(int)
    input2 = np.c_[X_1,X_2,X_3,X_4,X_5] #
    print ("appending done")
    return input2 

def prep_text(texts):
    text_sequences = tokenizer.texts_to_sequences(texts)
    return sequence.pad_sequences(text_sequences, maxlen=MAX_VOCAB)

def apply(model,df,param):
    input1 = prep_text(df['domain'])
    input2 = add_features(df)
    
    yhat = model.predict([input1,input2])
    y_ = (yhat > 0.5).astype("int32")
    output = pd.DataFrame()
    #output['domain']  = df['domain']
    #output['true_dga']  = df['is_dga'] 
    output['pred_dga']=y_.ravel()
    output['pred_dga_proba']=yhat.ravel()
    return output

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df,param))

## Stage 5 - save the model

In [ ]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [ ]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = tf.keras.models.load_model(MODEL_DIRECTORY + "pretrained_dga_apply_dga_model_dltk")
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

After implementing your fit, apply, save and load you can train your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| fit MLTKContainer algo=barebone s from feature_* into app:barebone_model<br>

Or apply your model:<br>
| makeresults count=10<br>
| streamstats c as i<br>
| eval s = i%3<br>
| eval feature_{s}=0<br>
| foreach feature_* [eval &lt;&lt;FIELD&gt;&gt;=random()/pow(2,31)]<br>
| apply barebone_model as the_meaning_of_life

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code